In [1]:
import pandas as pd
import numpy as np
from functools import partial

In [2]:
patient_characteristics_train = pd.read_csv('dataset/train/patient_characteristics.csv.gz')
chartevents_train = pd.read_csv('dataset/train/chartevents.csv.gz')
labevents_train = pd.read_csv('dataset/train/labevents.csv.gz')

In [3]:
len(set(patient_characteristics_train['ICUSTAY_ID'].unique()).intersection(set(chartevents_train['ICUSTAY_ID'].unique())))

7820

In [4]:
patient_characteristics_train['ICUSTAY_ID'].nunique()

7820

In [5]:
chartevents_train['ICUSTAY_ID'].nunique()

7820

In [6]:
patient_characteristics_train.shape

(7820, 22)

In [7]:
patient_characteristics_test = pd.read_csv('dataset/test/patient_characteristics.csv.gz')
chartevents_test = pd.read_csv('dataset/test/chartevents.csv.gz')
labevents_test = pd.read_csv('dataset/test/labevents.csv.gz')

In [8]:
len(set(patient_characteristics_test['ICUSTAY_ID'].unique()).intersection(set(chartevents_test['ICUSTAY_ID'].unique())))

1118

In [9]:
patient_characteristics_test['ICUSTAY_ID'].nunique()

1118

# Patient_Characteristics

In [10]:
patient_characteristics_train.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,...,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,HOSPITAL_EXPIRE_FLAG
0,371,274,130546,254851,MICU,MICU,12,12,2114-06-28 22:28:44,2114-07-07 18:01:16,...,2114-07-12 15:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,PROTESTANT QUAKER,MARRIED,BLACK/AFRICAN AMERICAN,0
1,379,283,109185,231490,MICU,MICU,15,15,2166-08-12 22:03:26,2166-09-12 14:41:42,...,2166-09-12 14:41:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,NaN,WIDOWED,WHITE,0
2,391,291,126219,246725,MICU,MICU,52,52,2107-09-13 22:43:01,2107-09-14 18:34:48,...,2107-09-16 18:50:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,SINGLE,WHITE,0
3,408,305,122211,224571,MICU,MICU,50,50,2127-06-19 23:50:45,2127-06-20 18:51:59,...,2127-07-16 13:58:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,PORT,CATHOLIC,MARRIED,WHITE,0
4,409,305,122211,232248,MICU,MICU,50,50,2127-07-03 20:02:34,2127-07-09 15:31:36,...,2127-07-16 13:58:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,PORT,CATHOLIC,MARRIED,WHITE,0


In [11]:
patient_characteristics_train.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 'INTIME', 'OUTTIME',
       'LOS', 'ADMITTIME', 'DISCHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'HOSPITAL_EXPIRE_FLAG'],
      dtype='object')

In [12]:
patient_characteristics_train.shape

(7820, 22)

## Number of HADM & Numer of ICUSTAY

In [13]:
def count_hadm_icustay(df):
    num_hadm = df.groupby('SUBJECT_ID')['HADM_ID'].count().rename('NUM_HADM')
    num_icustays = df.groupby('SUBJECT_ID')['ICUSTAY_ID'].count().rename('NUM_ICUSTAYS')
    
    df = pd.merge(df, num_hadm, on='SUBJECT_ID')
    df = pd.merge(df, num_icustays, on='SUBJECT_ID')
    
    return df

In [14]:
patient_characteristics_train = count_hadm_icustay(patient_characteristics_train)
patient_characteristics_test = count_hadm_icustay(patient_characteristics_test)

In [15]:
patient_characteristics_train.shape

(7820, 24)

## Length of Stay

In [16]:
patient_characteristics_train['LOS'].hist(bins=100)

In [17]:
def length_of_stay_quantiles(df, q_locs):
    for i in range(len(q_locs) - 1):
        q = q_locs[i]
        nq = q_locs[i + 1]
        df[f'LOS_q_{q:.2f}'] = df['LOS'].between(q, nq)
        
    return df

In [18]:
q_locs = []
for q in np.linspace(0, 0.8, 4):
    q_loc = patient_characteristics_train['LOS'].quantile(q)
    q_locs.append(q_loc)
    
q_locs.insert(0, 0)
q_locs.append(patient_characteristics_train['LOS'].max())

In [19]:
patient_characteristics_train = length_of_stay_quantiles(patient_characteristics_train, q_locs)
patient_characteristics_test = length_of_stay_quantiles(patient_characteristics_test, q_locs)

# Chartevents

In [20]:
chartevents_train.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,VALUE,NAME
0,85,112077,291697.0,2167-07-26 17:00:00,100.0,heartrate
1,85,112077,291697.0,2167-07-26 18:00:00,106.0,heartrate
2,85,112077,291697.0,2167-07-26 19:00:00,114.0,heartrate
3,85,112077,291697.0,2167-07-26 20:00:00,102.0,heartrate
4,85,112077,291697.0,2167-07-26 21:00:00,111.0,heartrate


In [21]:
quantiles = np.linspace(0.25, 0.75, 3)

quantile_functions = []

for quantile in quantiles:
    qf = partial(np.quantile, q=quantile)
    qf.__name__ = f'q{quantile:.2f}'
    quantile_functions.append(qf)

In [22]:
quantile_functions

[functools.partial(<function quantile at 0x11a438320>, q=0.25),
 functools.partial(<function quantile at 0x11a438320>, q=0.5),
 functools.partial(<function quantile at 0x11a438320>, q=0.75)]

In [23]:
chartevents_train_grouped = chartevents_train.groupby(['ICUSTAY_ID', 'NAME'])['VALUE'].agg(['min', 'max', 'mean', 'std', 'first', 'last', 'count', 'mad', 
                                                    *quantile_functions])

In [24]:
chartevents_train_grouped

min    max        mean        std  first   last  \
ICUSTAY_ID NAME                                                                
200021.0   arterial_bp_dia  53.0   77.0   65.538462   6.598368   56.0   58.0   
           arterial_bp_sys  93.0  126.0  111.423077  11.812445  104.0  110.0   
           heartrate        65.0  105.0   86.892857  11.275471   96.0   92.0   
           ni_bp_dia        60.0   78.0   71.000000   6.976150   77.0   70.0   
           ni_bp_sys        86.0  107.0   96.714286   8.440266  107.0   93.0   
...                          ...    ...         ...        ...    ...    ...   
299987.0   heartrate        56.0   72.0   64.476190   3.723158   68.0   67.0   
           ni_bp_dia        18.0   77.0   62.050000  14.687893   33.0   68.0   
           ni_bp_sys        89.0  152.0  125.100000  16.936103  136.0  128.0   
           resp_rate        16.0   32.0   23.350000   4.368247   32.0   29.0   
           saturation_o2    90.0   99.0   94.550000   2.282081   97.0   94.0   

                            count        mad   q0.25  q0.50   q0.75  
ICUSTAY_ID NAME                                                      
200021.0   arterial_bp_dia     26   5.426036   61.25   64.0   70.50  
           arterial_bp_sys     26  10.621302  101.50  114.5  122.75  
           heartrate           28   9.535714   80.25   86.0   96.50  
           ni_bp_dia            7   5.428571   67.00   71.0   77.00  
           ni_bp_sys            7   7.387755   91.00   93.0  104.50  
...                           ...        ...     ...    ...     ...  
299987.0   heartrate           21   2.934240   63.00   66.0   67.00  
           ni_bp_dia           20   9.945000   60.00   65.5   71.00  
           ni_bp_sys           20  12.310000  117.00  125.0  133.75  
           resp_rate           20   3.515000   20.50   24.0   26.00  
           saturation_o2       20   1.750000   94.00   94.5   96.00  

[46815 rows x 11 columns]

In [25]:
chartevents_test_grouped = chartevents_test.groupby(['ICUSTAY_ID', 'NAME'])['VALUE'].agg(['min', 'max', 'mean', 'std', 'first', 'last', 'count', 'mad', 
                                                    *quantile_functions])

# Labevents

In [26]:
labevents_train.head()

,SUBJECT_ID,HADM_ID,CHARTTIME,VALUE,NAME,ICUSTAY_ID
0,3,145834.0,2101-10-22 04:31:00,7.40,ph,211552.0
1,3,145834.0,2101-10-22 07:13:00,7.37,ph,211552.0
2,3,145834.0,2101-10-22 10:16:00,7.37,ph,211552.0
3,3,145834.0,2101-10-22 11:21:00,7.39,ph,211552.0
4,3,145834.0,2101-10-22 13:02:00,7.39,ph,211552.0


In [27]:
labevents_train_grouped = labevents_train.groupby(['ICUSTAY_ID', 'NAME'])['VALUE'].agg(['min', 'max', 'mean', 'std', 'first', 'last', 'count', 'mad', 
                                                    *quantile_functions])

labevents_test_grouped = labevents_test.groupby(['ICUSTAY_ID', 'NAME'])['VALUE'].agg(['min', 'max', 'mean', 'std', 'first', 'last', 'count', 'mad', 
                                                    *quantile_functions])

In [28]:
def unravel_groups(df):
    df = df.unstack()
    df.columns = ['_'.join(val for val in col) for col in df.columns]
    return df


In [29]:
chartevents_train_grouped = unravel_groups(chartevents_train_grouped)
chartevents_test_grouped = unravel_groups(chartevents_test_grouped)

labevents_train_grouped = unravel_groups(labevents_train_grouped)
labevents_test_grouped = unravel_groups(labevents_test_grouped)

In [30]:
chartevents_train_grouped.shape

(7820, 77)

In [31]:
labevents_train_grouped.shape

(7820, 143)

In [32]:
chartevents_train_grouped.index.nunique()

7820

In [33]:
patient_characteristics_train['ICUSTAY_ID'].nunique()

7820

In [34]:
labevents_train_grouped.index.nunique()

7820

In [35]:
len(set(chartevents_train_grouped.index).intersection(set(patient_characteristics_train['ICUSTAY_ID'])))

7820

In [36]:
pd.merge(chartevents_train_grouped, patient_characteristics_train, left_index=True, right_on='ICUSTAY_ID').shape

(7820, 106)

# Merging & Model Preparation

In [65]:
df_model_train = pd.merge(
    pd.merge(chartevents_train_grouped, patient_characteristics_train, on='ICUSTAY_ID'),
    labevents_train_grouped, on='ICUSTAY_ID')

In [66]:
df_model_test = pd.merge(
    pd.merge(chartevents_test_grouped, patient_characteristics_test, on='ICUSTAY_ID'),
    labevents_test_grouped, on='ICUSTAY_ID')

In [67]:
df_model_train.shape, df_model_test.shape

((7820, 249), (1118, 249))

In [68]:
to_drop = ['ROW_ID',
           'SUBJECT_ID',
           'HADM_ID',
           'INTIME',
           'OUTTIME',
           'ADMITTIME',
           'DISCHTIME']

diagnoses_train = pd.read_csv('dataset/train/diagnoses_train.csv.gz')
diagnoses_test = pd.read_csv('dataset/test/diagnoses_test.csv.gz')

df_model_train = df_model_train.drop(to_drop, axis=1)
df_model_test = df_model_test.drop(to_drop, axis=1)

In [77]:
diagnoses_train = diagnoses_train.sort_values('ICUSTAY_ID')
df_model_train = df_model_train.sort_values('ICUSTAY_ID')

diagnoses_test = diagnoses_test.sort_values('ICUSTAY_ID')
df_model_test = df_model_test.sort_values('ICUSTAY_ID')

In [81]:
categorical_columns = [
 'ETHNICITY',
 'INSURANCE',
 'RELIGION',
 'ADMISSION_TYPE',
 'ADMISSION_LOCATION',
 'MARITAL_STATUS',
 'LANGUAGE',
 'FIRST_CAREUNIT',
 'DISCHARGE_LOCATION',
 'FIRST_WARDID',
 'LAST_WARDID',
 'LAST_CAREUNIT']

In [82]:
def to_numeric_df(df):
    for col in df.columns:
        df[col] = pd.to_numeric(df[col])
    return df

In [83]:
df_model_train = pd.get_dummies(df_model_train, columns=categorical_columns, drop_first=True, dummy_na=True)

df_model_test = pd.get_dummies(df_model_test, columns=categorical_columns, drop_first=True, dummy_na=True)

In [84]:
df_model_train.shape

(7820, 394)

In [85]:
column_intersection = list(set(df_model_train.columns).intersection(set(df_model_test.columns)))

df_model_train = df_model_train[column_intersection]
df_model_test = df_model_test[column_intersection]

In [86]:
df_model_train.shape

(7820, 347)

In [87]:
df_model_train = to_numeric_df(df_model_train)
df_model_test = to_numeric_df(df_model_test)

In [88]:
df_model_train = df_model_train.fillna(df_model_train.mean())

df_model_test = df_model_test.fillna(df_model_test.mean())

# Modelling

In [89]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

X_train = df_model_train.drop('ICUSTAY_ID', axis=1)
X_test = df_model_test.drop('ICUSTAY_ID', axis=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [91]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(diagnoses_train['DIAGNOSIS'])
y_test = encoder.transform(diagnoses_test['DIAGNOSIS'])

In [92]:
from sklearn.ensemble import GradientBoostingClassifier

In [93]:
model = GradientBoostingClassifier()

In [94]:
y_train.shape, X_train.shape

((7820,), (7820, 346))

In [102]:
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [103]:
y_pred = model.predict(X_train)

In [104]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [105]:
print(f'Accuracy: {accuracy_score(y_train, y_pred)}')


cm = pd.DataFrame(confusion_matrix(y_train, y_pred))

cm.columns = encoder.inverse_transform(cm.columns)
cm.index = encoder.inverse_transform(cm.index)

Accuracy: 0.829156010230179


In [106]:
cm

,ALTERED MENTAL STATUS,CONGESTIVE HEART FAILURE,CORONARY ARTERY DISEASE,DIABETIC KETOACIDOSIS,HYPOTENSION,INTRACRANIAL HEMORRHAGE,PNEUMONIA,SEPSIS,STROKE,SUBARACHNOID HEMORRHAGE
ALTERED MENTAL STATUS,403,22,1,4,5,28,115,51,10,10
CONGESTIVE HEART FAILURE,3,712,40,0,0,4,95,18,0,0
CORONARY ARTERY DISEASE,1,19,1644,0,0,0,2,7,0,2
DIABETIC KETOACIDOSIS,3,1,0,375,0,0,5,5,1,0
HYPOTENSION,9,24,3,6,219,4,61,65,0,0
INTRACRANIAL HEMORRHAGE,6,7,0,0,0,519,16,11,13,13
PNEUMONIA,24,59,7,4,7,17,1183,113,4,1
SEPSIS,13,41,8,6,9,15,186,809,2,3
STROKE,2,6,2,0,0,38,8,5,296,6
SUBARACHNOID HEMORRHAGE,5,2,1,0,0,40,5,4,3,324


# Submission

In [119]:
y_pred_test = model.predict(X_test)

In [120]:
diagnoses_test = encoder.inverse_transform(y_pred_test)

In [121]:
submission_df = pd.concat([df_model_test['ICUSTAY_ID'], pd.Series(diagnoses_test, name='DIAGNOSIS_PRED')], axis=1)

In [124]:
submission_df.to_csv('submission_name.csv', index=False)